In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

def get_title(soup):
    try:
        title = soup.find('h1' , attrs={'class':"product-description__name"}) 
        
        title_value = title.text
        
        title_string = title_value.strip()
        
    except AttributeError:
        title_string = ""
    
    return title_string


def get_Product_code(soup):
    try :
        product_code = soup.find('div' , attrs={'class':"product-details--product-code"}).text.strip()
        
    except AttributeError:
        product_code = ""
        
    return product_code



def get_price(soup):
    try:
        price = soup.find('span' , attrs={'class':"product-description__price"}).string.strip()
        
    except AttributeError:
        price = ""
        
    return price
            
        
        
if __name__ == '__main__':

    # add your user agent 
    headers = ({'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36'})
    # The webpage URL
    urls = 'https://www.newlook.com/row/mens/clothing/c/row-mens-clothing#/?q=:relevance&page=0&sort=relevance&content=false'

    # HTTP Request
    webpage = requests.get(urls, headers=headers)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all('a' , attrs= {'class':'plp-carousel__img-link'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))

    d = {"title":[], "price":[], "product_code":[]}
    
    # Loop for extracting product details from each link 
    for link in links_list:
        new_webpage = requests.get("https://www.newlook.com" + link, headers=headers)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['product_code'].append(get_Product_code(new_soup))

    
    
    
    
    
# Define a function to scrape product details from a page
def scrape_page(url):
    webpage = requests.get(url, headers=headers)
    soup = BeautifulSoup(webpage.content, "html.parser")

    links = soup.find_all('a', attrs={'class': 'plp-carousel__img-link'})

    for link in links:
        new_webpage = requests.get("https://www.newlook.com" + link.get('href'), headers=headers)
        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        title = get_title(new_soup)
        price = get_price(new_soup)
        product_code = get_Product_code(new_soup)

        d['title'].append(title)
        d['price'].append(price)
        d['product_code'].append(product_code)

# The initial URL
base_url = urls
page_number = 1

while True:
    # Construct the URL for the current page
    current_page_url = f'{base_url}?page={page_number}'

    # Scrape product details from the current page
    scrape_page(current_page_url)

    # Check if there is a next page by inspecting the website's structure
    # If there's no next page, break the loop
    next_button = soup.find('span', class_='ng-scope')  # Example class name for the "Next" button

    if next_button:
        # There is a next page, so continue to the next iteration
        page_number += 1
    else:
        # No "Next" button found, break the loop
        break
    
    

    # Increment the page number for the next iteration
    page_number += 1

# Create a DataFrame from the collected data
newlook_df = pd.DataFrame.from_dict(d)
newlook_df['title'].replace('', np.nan, inplace=True)
newlook_df = newlook_df.dropna(subset=['title'])
newlook_df.to_csv("newlook_data.csv", header=True, index=False)






In [17]:

from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

# Define functions to extract product details
def get_title(soup):
    try:
        title = soup.find('h1', attrs={'class': "product-description__name"})
        title_value = title.text
        title_string = title_value.strip()
    except AttributeError:
        title_string = ""
    return title_string

def get_Product_code(soup):
    try:
        product_code = soup.find('div', attrs={'class': "product-details--product-code"}).text.strip()
    except AttributeError:
        product_code = ""
    return product_code

def get_price(soup):
    try:
        price = soup.find('span', attrs={'class': "product-description__price"}).string.strip()
    except AttributeError:
        price = ""
    return price

if __name__ == '__main__':
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36 Edg/116.0.1938.69'}
    base_url = 'https://www.newlook.com/row/womens/clothing/dresses/c/row-womens-clothing-dresses#/?q=:relevance&page={page_number}&sort=relevance&content=false'
    page_number = 0
    
    final_data = {"title": [], "price": [], "product_code": []}
    
    while True:
        current_page_url = f'{base_url}?page={page_number}'
        webpage = requests.get(current_page_url, headers=headers)
        soup = BeautifulSoup(webpage.content, "html.parser")
        links = soup.find_all('a', attrs={'class': 'plp-carousel__img-link'})
        
        if not links:
            # No more products on this page, break the loop
            break
        
        for link in links:
            product_url = "https://www.newlook.com" + link.get('href')
            product_webpage = requests.get(product_url, headers=headers)
            product_soup = BeautifulSoup(product_webpage.content, "html.parser")
            
            title = get_title(product_soup)
            price = get_price(product_soup)
            product_code = get_Product_code(product_soup)
            
            final_data['title'].append(title)
            final_data['price'].append(price)
            final_data['product_code'].append(product_code)
        
        page_number += 1
    
    # Create a DataFrame from the collected data
    newlook_df = pd.DataFrame.from_dict(final_data)
    newlook_df['title'].replace('', np.nan, inplace=True)
    newlook_df = newlook_df.dropna(subset=['title'])
    newlook_df.to_csv("newlook_data.csv", header=True, index=False)


In [ ]:
from selectolax.parser import HTMLParser
import httpx
from dataclasses import dataclass, asdict


@dataclass
class product:
    product_code: str
    price: str
    title: str


def get_html(page):
    url = f'https://www.newlook.com/row/womens/clothing/c/row-womens-clothing#/?q=:relevance&page={page}&sort=relevance&content=false'
    resp = httpx.get(url)
    return HTMLParser(resp.text)


def main():
    html = get_html(1)
    print(html)


if __name__ == '__main__':
    main()


In [24]:

from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

# Define functions to extract product details
def get_title(soup):
    try:
        title = soup.find('h1', attrs={'class': "product-description__name"})
        title_value = title.text
        title_string = title_value.strip()
    except AttributeError:
        title_string = ""
    return title_string

def get_Product_code(soup):
    try:
        product_code = soup.find('div', attrs={'class': "product-details--product-code"}).text.strip()
    except AttributeError:
        product_code = ""
    return product_code

def get_price(soup):
    try:
        price = soup.find('span', attrs={'class': "product-description__price"}).string.strip()
    except AttributeError:
        price = ""
    return price

if __name__ == '__main__':
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36 Edg/116.0.1938.69'}
    base_url = 'https://www.newlook.com/row/womens/clothing/dresses/c/row-womens-clothing-dresses#/?q=:relevance&page={page_number}&sort=relevance&content=false'
    page_number = 1
    
    final_data = {"title": [], "price": [], "product_code": []}
    
    while True:
        current_page_url = f'{base_url}?page={page_number}'
        webpage = requests.get(current_page_url, headers=headers)
        soup = BeautifulSoup(webpage.content, "html.parser")
        links = soup.find_all('a', attrs={'class': 'plp-carousel__img-link'})
        
        if not links:
            # No more products on this page, break the loop
            break
        
        for link in links:
            product_url = "https://www.newlook.com" + link.get('href')
            product_webpage = requests.get(product_url, headers=headers)
            product_soup = BeautifulSoup(product_webpage.content, "html.parser")
            
            title = get_title(product_soup)
            price = get_price(product_soup)
            product_code = get_Product_code(product_soup)
            
            final_data['title'].append(title)
            final_data['price'].append(price)
            final_data['product_code'].append(product_code)
        
        # Check for the absence of a "Next Page" button
        next_button = soup.find('a', attrs={'class': 'next-page'})
        if not next_button:
            # No more next page, break the loop
            break
        
        page_number += 1
        
    
    # Create a DataFrame from the collected data
    newlook_df = pd.DataFrame.from_dict(final_data)
    newlook_df['title'].replace('', np.nan, inplace=True)
    newlook_df = newlook_df.dropna(subset=['title'])
    newlook_df.to_csv("newlook_data.csv", header=True, index=False)


In [25]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import time

# Inside your loop:


def get_title(soup):
    try:
        title = soup.find('h1' , attrs={'class':"product-description__name"}) 
        
        title_value = title.text
        
        title_string = title_value.strip()
        
    except AttributeError:
        title_string = ""
    
    return title_string


def get_Product_code(soup):
    try :
        product_code = soup.find('div' , attrs={'class':"product-details--product-code"}).text.strip()
        
    except AttributeError:
        product_code = ""
        
    return product_code



def get_price(soup):
    try:
        price = soup.find('span' , attrs={'class':"product-description__price"}).string.strip()
        
    except AttributeError:
        price = ""
        
    return price
            
        
        
if __name__ == '__main__':

    # add your user agent 
    headers = ({'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36'})
    # The webpage URL
    urls = 'https://www.newlook.com/row/boys/clothing/c/row-boys-clothing#/?q=:relevance&page=1&sort=relevance&content=false'

    # HTTP Request
    webpage = requests.get(urls, headers=headers)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all('a' , attrs= {'class':'plp-carousel__img-link'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))

    d = {"title":[], "price":[], "product_code":[]}
    
    # Loop for extracting product details from each link 
    for link in links_list:
        new_webpage = requests.get("https://www.newlook.com" + link, headers=headers)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['product_code'].append(get_Product_code(new_soup))

    
    
    
    
    
# Define a function to scrape product details from a page
def scrape_page(url):
    webpage = requests.get(url, headers=headers)
    soup = BeautifulSoup(webpage.content, "html.parser")

    links = soup.find_all('a', attrs={'class': 'plp-carousel__img-link'})

    for link in links:
        new_webpage = requests.get("https://www.newlook.com" + link.get('href'), headers=headers)
        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        title = get_title(new_soup)
        price = get_price(new_soup)
        product_code = get_Product_code(new_soup)

        d['title'].append(title)
        d['price'].append(price)
        d['product_code'].append(product_code)


# The initial URL
base_url = urls
page_number = 1

while True:
    # Construct the URL for the current page
    
    current_page_url = f'{base_url}?page={page_number}'

    # Scrape product details from the current page
    scrape_page(current_page_url)
    time.sleep(2) 

    # Check if there is a next page by inspecting the website's structure
    # If there's no next page, break the loop
    next_button = soup.find('span', class_='ng-scope')  # Example class name for the "Next" button

    if next_button:
        # There is a next page, so continue to the next iteration
        page_number += 1
    else:
        # No "Next" button found, break the loop
        break

# ... (rest of the code)



# Create a DataFrame from the collected data
newlook_df = pd.DataFrame.from_dict(d)
newlook_df['title'].replace('', np.nan, inplace=True)
newlook_df = newlook_df.dropna(subset=['title'])
newlook_df.to_csv("newlook_data.csv", header=True, index=False)




